In [1]:
#@title Setup YOLOV5
# !git clone https://github.com/ultralytics/yolov5  # clone yolov5
# !git clone https://github.com/cv516Buaa/tph-yolov5 # clone tph-yolov5
!git clone https://github.com/smbanx/ECS271-yolov5
# %cd yolov5
%cd ECS271-yolov5
%pip install -qr requirements.txt comet_ml  # install

import torch
import utils
display = utils.notebook_init()  # checks

YOLOv5 🚀 43480da Python-3.10.12 torch-2.3.0+cu121 CUDA:0 (NVIDIA A100-SXM4-40GB, 40514MiB)


Setup complete ✅ (12 CPUs, 83.5 GB RAM, 30.0/201.2 GB disk)


In [2]:
#@title Mount Data from Google Drive
# %cd ./tph-yolov5
# %pip install -r requirements.txt
%cd ..

/content


In [3]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
#@title Create Train/Val Sets (NOTE: DO NOT RERUN)
%cd ./drive/MyDrive/ECS271/
%ls
# !python customTrainVal.py

/content/drive/MyDrive/ECS271
common.py                 custom_val_2500/         project.yaml             yolov5l-xs-2.pt
custom_train/             custom_val_2500.cache    train/                   yolov5l-xs-tph.yaml
custom_train_10000/       custom_val.cache         train_class_lookup.json  yolov5m_soda.yaml
custom_train_10000.cache  dataloaders_modified.py  train-results/           yolov5n_soda.yaml
custom_train_4000/        detection-results/       val/                     yolov5s_soda.yaml
custom_train_4000.cache   heli_train_10000/        val_class_lookup.json    yolov5x-seg_tf.yaml
custom_train.cache        heli_train_10000.cache   val-results/             yolov5x_soda.yaml
customTrainVal.py         heli_val_2500/           yolov5l_soda.yaml        yolov5x_tf.yaml
custom_val/               heli_val_2500.cache      yolov5l_tf.yaml
custom_val_1000/          params.txt               yolov5l-tph-plus.yaml
custom_val_1000.cache     projectv2.yaml           yolov5l-xs-1.pt


In [ ]:
import os
import random
import shutil
import glob
from datetime import datetime

# function for removing duplicates from a list
def unique(lst):
    used = set()
    new_lst = []
    for elem in lst:
        if elem not in used:
            new_lst.append(elem)
            used.add(elem)
    return new_lst

In [ ]:
# read params file
train_path = "./train/"
train_dir_list = os.listdir(train_path)
train_dir_list = unique([elem[:-4] for elem in train_dir_list])
print(len(train_dir_list))

val_path = "./val/"
val_dir_list = os.listdir(val_path)
val_dir_list = unique([elem[:-4] for elem in val_dir_list])
print(len(val_dir_list))

# new_train_path = "../../../train/"
new_train_path = "./comprehensive_train_10000/" #
if not os.path.exists(new_train_path):
  os.makedirs(new_train_path)

# new_val_path = "../../../val/"
new_val_path = "./comprehensive_val_2500/" #
if not os.path.exists(new_val_path):
  os.makedirs(new_val_path)

n_train = 10000 #
n_train = len(train_dir_list) if n_train == -1 else n_train
n_val = 2500 #
n_val = len(val_dir_list) if n_val == -1 else n_val

randomize = True #
random_seed = 42 #

if random_seed == -1:
  random.seed(datetime.now().timestamp())
else:
  random.seed(random_seed)

# load json files
import json

# Opening JSON file
f_train = open('train_class_lookup.json')
f_val = open("val_class_lookup.json")

# return JSON object as a dictionary
train_data = json.load(f_train)
val_data = json.load(f_val)

# Close file
f_train.close()
f_val.close()

OSError: [Errno 5] Input/output error: './train/'

In [ ]:
# empty custom train/val folders
files = glob.glob(new_train_path + "*")
for f in files:
    os.remove(f)
files = glob.glob(new_val_path + "*")
for f in files:
    os.remove(f)

# randomize if necessary
if randomize:
    random.shuffle(train_dir_list)
    random.shuffle(val_dir_list)

count = 0

# copy files to new location
# train
for category in train_data.keys():
  for i in range(1000):
    count += 1
    if count % 500 == 0:
      print("Train: " + str(count))
    shutil.copyfile(train_path + train_data[category][i] + ".jpg", new_train_path + train_data[category][i] + ".jpg")
    shutil.copyfile(train_path + train_data[category][i] + ".txt", new_train_path + train_data[category][i] + ".txt")

remaining_train = n_train - count
for i in range(remaining_train):
    count += 1
    if count % 500 == 0:
      print("Train: " + str(count))
    shutil.copyfile(train_path + train_dir_list[i] + ".jpg", new_train_path + train_dir_list[i] + ".jpg")
    shutil.copyfile(train_path + train_dir_list[i] + ".txt", new_train_path + train_dir_list[i] + ".txt")

# val
count = 0
for category in val_data.keys():
  for i in range(250):
    count += 1
    if count % 500 == 0:
      print("Val: " + str(count))
    shutil.copyfile(train_path + val_data[category][i] + ".jpg", new_train_path + val_data[category][i] + ".jpg")
    shutil.copyfile(train_path + val_data[category][i] + ".txt", new_train_path + val_data[category][i] + ".txt")

remaining_val = n_val - count
for i in range(remaining_val):
    if count % 500 == 0:
      print("Val: " + str(count))
    shutil.copyfile(val_path + val_dir_list[i] + ".jpg", new_val_path + val_dir_list[i] + ".jpg")
    shutil.copyfile(val_path + val_dir_list[i] + ".txt", new_val_path + val_dir_list[i] + ".txt")

Train: 0
Train: 500
Train: 1000
Train: 1500
Train: 2000
Train: 2500
Train: 3000
Train: 3500
Train: 4000
Train: 4500
Train: 5000
Train: 5500
Train: 6000
Train: 6500
Train: 7000
Train: 7500
Train: 8000
Train: 8500
Train: 9000
Train: 9500
Val: 0
Val: 500
Val: 1000
Val: 1500
Val: 2000


In [ ]:
%cd ./drive/MyDrive/ECS271/
import os

new_train_path = "./heli_train_10000/" #
n_train = 10000
new_val_path = "./heli_val_2500/" #
n_val = 2500
new_train_dir_list = os.listdir(new_train_path)
# assert(len(new_train_dir_list) == 2 * n_train)
print(len(new_train_dir_list))
new_val_dir_list = os.listdir(new_val_path)
# assert(len(new_val_dir_list) == 2 * n_val)
print(len(new_val_dir_list))
%cd ../../..

/content/drive/MyDrive/ECS271
20001
5000
/content


In [ ]:
#@title Copy YAML Files (Note: only run if cloning original YOLO repo)
%cd ./drive/MyDrive/ECS271/
import shutil
shutil.copyfile("./project.yaml", "../../../yolov5/data/project.yaml")
shutil.copyfile("./projectv2.yaml", "../../../yolov5/data/projectv2.yaml")
shutil.copyfile("./yolov5n_soda.yaml", "../../../yolov5/models/yolov5n.yaml")
shutil.copyfile("./yolov5s_soda.yaml", "../../../yolov5/models/yolov5s.yaml")
shutil.copyfile("./yolov5m_soda.yaml", "../../../yolov5/models/yolov5m.yaml")
shutil.copyfile("./yolov5l_tf.yaml", "../../../yolov5/models/yolov5l.yaml")
shutil.copyfile("./yolov5x_tf.yaml", "../../../yolov5/models/yolov5x.yaml")
shutil.copyfile("./yolov5x-seg_tf.yaml", "../../../yolov5/models/segment/yolov5x-seg.yaml")
shutil.copyfile("./dataloaders_modified.py", "../../../yolov5/utils/dataloaders.py")
# shutil.copyfile("./yolov5l-tph-plus.yaml", "../../../yolov5/models/yolov5l-tph-plus.yaml")
# shutil.copyfile("./yolov5l-tph-plus.yaml", "../../../yolov5/models/yolov5l-tph-plus.yaml")
# shutil.copyfile("./yolov5l-xs-tph.yaml", "../../../yolov5/models/yolov5l-xs-1.yaml")
# shutil.copyfile("./yolov5l-xs-tph.yaml", "../../../yolov5/models/yolov5l-xs-2.yaml")
# shutil.copyfile("./yolov5l-xs-1.pt", "../../../yolov5/yolov5l-xs-1.pt")
# shutil.copyfile("./yolov5l-xs-2.pt", "../../../yolov5/yolov5l-xs-2.pt")
# shutil.copyfile("./common.py", "../../../yolov5/models/common.py")
'''
shutil.copyfile("./project.yaml", "../../../tph-yolov5/data/project.yaml")
shutil.copyfile("./yolov5l-xs-1.pt", "../../../tph-yolov5/yolov5l-tph-plus.pt")
'''

/content/drive/MyDrive/ECS271


'\nshutil.copyfile("./project.yaml", "../../../tph-yolov5/data/project.yaml")\nshutil.copyfile("./yolov5l-xs-1.pt", "../../../tph-yolov5/yolov5l-tph-plus.pt")\n'

In [5]:
# %cd ../../../yolov5
# %cd ../../../tph-yolov5
# %cd ECS271-yolov5
%cd ../../../ECS271-yolov5

/content/ECS271-yolov5


In [7]:
#@title Train YOLOv5 on SODA-A for 3 epochs
!TORCH_USE_CUDA_DSA=1 python train.py --img 800 --batch 8 --epochs 3 --data project.yaml --cfg yolov5x.yaml --bbox_interval 1 --weights yolov5x.pt --cache --name tfx_soda_heli_
# !TORCH_USE_CUDA_DSA=1 python train.py --img 800 --batch 4 --epochs 3 --data project.yaml --cfg yolov5x.yaml --bbox_interval 1 --weights yolov5x.pt --name tfx_soda_heli_
# !TORCH_USE_CUDA_DSA=1 python train.py --img 800 --batch 16 --epochs 3 --data project.yaml --cfg yolov5l-tph-plus.yaml --bbox_interval 1 --weights yolov5l-tph-plus.pt --name 5l-tph-plus_soda_10k_2.5k_

2024-06-10 04:13:06.771957: E external/local_xla/xla/stream_executor/cuda/cuda_dnn.cc:9261] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
2024-06-10 04:13:06.772007: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:607] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
2024-06-10 04:13:06.773578: E external/local_xla/xla/stream_executor/cuda/cuda_blas.cc:1515] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered
train: weights=yolov5x.pt, cfg=yolov5x.yaml, data=project.yaml, hyp=data/hyps/hyp.scratch-low.yaml, epochs=3, batch_size=8, imgsz=800, rect=False, resume=False, nosave=False, noval=False, noautoanchor=False, noplots=False, evolve=None, evolve_population=data/hyps, resume_evolve=None, bucket=, cache=ram, image_weights=False, device=, multi_scale=False, single_cls=

KeyboardInterrupt: 

In [ ]:
import os
import shutil

results_path = "./runs/train/tfx_soda_10k_2.5k_3/"
new_results_path = "../drive/MyDrive/ECS271/train-results/tfx_soda_10k_2.5k_modified/"
if not os.path.exists(new_results_path):
  os.makedirs(new_results_path)
if not os.path.exists(new_results_path + "weights/"):
  os.makedirs(new_results_path + "weights/")
results_dir_list = os.listdir(results_path)
for result in results_dir_list:
  if result != "weights":
    shutil.copyfile(results_path + result, new_results_path + result)
weights_dir_list = os.listdir(results_path + "weights/")
for weight in weights_dir_list:
  shutil.copyfile(results_path + "weights/" + weight, new_results_path + "weights/" + weight)

In [ ]:
#@title Validate Model on Project Validation Set
# !python val.py --weights runs/train/x_soda_10k_2.5k_2/weights/best.pt --data projectv2.yaml --img 640 --half --name x_soda_10k_2.5k_
!python val.py --weights ../drive/MyDrive/ECS271/train-results/x_soda_10k_2.5k_2/weights/best.pt --data projectv2.yaml --img 640 --half --name x_soda_10k_2.5k_

val: data=/content/yolov5/data/projectv2.yaml, weights=['../drive/MyDrive/ECS271/train-results/x_soda_10k_2.5k_2/weights/best.pt'], batch_size=32, imgsz=640, conf_thres=0.001, iou_thres=0.6, max_det=300, task=val, device=, workers=8, single_cls=False, augment=False, verbose=False, save_txt=False, save_hybrid=False, save_conf=False, save_json=False, project=runs/val, name=x_soda_10k_2.5k_, exist_ok=False, half=True, dnn=False
YOLOv5 🚀 v7.0-314-g60dde7f1 Python-3.10.12 torch-2.3.0+cu121 CUDA:0 (NVIDIA L4, 22700MiB)

Fusing layers... 
Model summary: 322 layers, 86227246 parameters, 0 gradients, 203.9 GFLOPs
val: Scanning /content/drive/MyDrive/ECS271/custom_val_2500.cache... 2500 images, 1228 backgrounds, 0 corrupt: 100% 2500/2500 [00:00<?, ?it/s]
val: WARNING ⚠️ /content/drive/MyDrive/ECS271/custom_val_2500/01961__800__3250___0.jpg: 4 duplicate labels removed
val: WARNING ⚠️ /content/drive/MyDrive/ECS271/custom_val_2500/01961__800__650___650.jpg: 21 duplicate labels removed
val: WARNING 

In [ ]:
import os
import shutil

results_path = "./runs/val/x_soda_10k_2.5k_/"
new_results_path = "../drive/MyDrive/ECS271/val-results/x_soda_10k_2.5k_/"
if not os.path.exists(new_results_path):
  os.makedirs(new_results_path)
results_dir_list = os.listdir(results_path)
for result in results_dir_list:
    shutil.copyfile(results_path + result, new_results_path + result)

In [ ]:
#@title mAP
%cd ..
!git clone https://github.com/Cartucho/mAP # clone mAP github
%cd yolov5

/content
Cloning into 'mAP'...
remote: Enumerating objects: 908, done.
remote: Counting objects: 100% (150/150), done.
remote: Compressing objects: 100% (25/25), done.
remote: Total 908 (delta 135), reused 125 (delta 125), pack-reused 758
Receiving objects: 100% (908/908), 14.70 MiB | 13.32 MiB/s, done.
Resolving deltas: 100% (329/329), done.


In [ ]:
%%capture
!python -mpip install -U matplotlib
!python -mpip install -U opencv-python

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 8.3/8.3 MB 63.1 MB/s eta 0:00:00
  Attempting uninstall: matplotlib
    Found existing installation: matplotlib 3.7.1
    Uninstalling matplotlib-3.7.1:
      Successfully uninstalled matplotlib-3.7.1
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 62.2/62.2 MB 28.2 MB/s eta 0:00:00
  Attempting uninstall: opencv-python
    Found existing installation: opencv-python 4.8.0.76
    Uninstalling opencv-python-4.8.0.76:
      Successfully uninstalled opencv-python-4.8.0.76


In [ ]:
%%capture
# !python detect.py --weights runs/train/exp8/weights/best.pt --source ../val --save-txt --save-conf --project ../detection-results/ --name exp --conf-thres 0.1
!python detect.py --weights runs/train/x_soda_10k_2.5k_2/weights/best.pt --source ../drive/MyDrive/ECS271/custom_val --save-txt --save-conf --project ../drive/MyDrive/ECS271/detection-results/ --name x_soda_10k_2.5k_
#!python detect.py --weights runs/train/l_soda_10k_2.5k_3/weights/best.pt --source ../drive/MyDrive/ECS271/custom_val --save-txt --save-conf --project ../detection-results/ --name l_soda_10k_2.5k_
#!python detect.py --weights runs/train/m_soda_10k_2.5k_3/weights/best.pt --source ../drive/MyDrive/ECS271/custom_val --save-txt --save-conf --project ../detection-results/ --name m_soda_10k_2.5k_
#!python detect.py --weights runs/train/s_soda_10k_2.5k_3/weights/best.pt --source ../drive/MyDrive/ECS271/custom_val --save-txt --save-conf --project ../detection-results/ --name s_soda_10k_2.5k_
#!python detect.py --weights runs/train/n_soda_10k_2.5k_3/weights/best.pt --source ../drive/MyDrive/ECS271/custom_val --save-txt --save-conf --project ../detection-results/ --name n_soda_10k_2.5k_
#!python detect.py --weights runs/train/tf_soda_10k_2.5k_3/weights/best.pt --source ../drive/MyDrive/ECS271/custom_val --save-txt --save-conf --project ../detection-results/ --name tf_soda_10k_2.5k_
%cd ..

In [ ]:
import os
import shutil
import glob

detection_results_path = "./mAP/input/detection-results/"
ground_truth_path = "./mAP/input/ground-truth/"
images_path = "./mAP/input/images-optional/"

# delete files currently in path
files = glob.glob(detection_results_path + "*")
for f in files:
    os.remove(f)
files = glob.glob(ground_truth_path + "*")
for f in files:
    os.remove(f)
files = glob.glob(images_path + "*")
for f in files:
    os.remove(f)

In [ ]:
# function for removing duplicates from a list
def unique(lst):
    used = set()
    new_lst = []
    for elem in lst:
        if elem not in used:
            new_lst.append(elem)
            used.add(elem)
    return new_lst

detection_results_source = "./detection-results/exp4/labels/"
ground_truth_source = "./drive/MyDrive/ECS271/custom_val/"

# copy detection results
detection_results_list = os.listdir(detection_results_source)
detection_results_list = unique([elem[:-4] for elem in detection_results_list])
detection_results_set = set(detection_results_list)
n_results = len(detection_results_list)
for i in range(n_results):
    shutil.copyfile(detection_results_source + detection_results_list[i] + ".txt", detection_results_path + detection_results_list[i] + ".txt")

# copy ground truth files
ground_truth_list = os.listdir(ground_truth_source)
ground_truth_list = unique([elem[:-4] for elem in ground_truth_list])
n_truth = len(ground_truth_list)
for i in range(n_truth):
    shutil.copyfile(ground_truth_source + ground_truth_list[i] + ".txt", ground_truth_path + ground_truth_list[i] + ".txt")
    shutil.copyfile(ground_truth_source + ground_truth_list[i] + ".jpg", images_path + ground_truth_list[i] + ".jpg")
    if(ground_truth_list[i] not in detection_results_set):
      with open(detection_results_source + ground_truth_list[i] + ".txt", 'w') as file:
        pass

In [ ]:
import cv2
!pip install opencv-python-headless

In [ ]:
%cd ./mAP/
!python main.py
%cd ..

/content/mAP
qt.qpa.xcb: could not connect to display 
qt.qpa.plugin: Could not load the Qt platform plugin "xcb" in "/usr/local/lib/python3.10/dist-packages/cv2/qt/plugins" even though it was found.
This application failed to start because no Qt platform plugin could be initialized. Reinstalling the application may fix this problem.

Available platform plugins are: xcb.

/content
